In [1]:
import numpy as np

# 2 im2col関数

In [2]:
def im2col(x, fil_size, y_size, stride, pad):
    x_b, x_c, x_h, x_w = x.shape
    fil_h, fil_w = fil_size, fil_size
    y_h, y_w = y_size, y_size
    index = -1
    
    x_pad = np.pad(x, [(0, 0), (0, 0), (pad, pad), (pad, pad)], "constant")
    x_col = np.zeros((fil_h*fil_w, x_b, x_c, y_h, y_w))
    
    for h in range(fil_h):
        h2 = h + y_h*stride
        for w in range(fil_w):
            index += 1
            w2 = w + y_w*stride
            x_col[index,:,:,:,:] = x_pad[:,:,h:h2:stride,w:w2:stride]
    x_col = x_col.transpose(2,0,1,3,4).reshape(x_c*fil_h*fil_w, x_b*y_h*y_w)
    
    return x_col

In [3]:
x = np.arange(144).reshape(3,3,4,4)
x_col = im2col(x,3,2,1,0)
x_col.shape

(27, 12)

In [4]:
w = np.arange(54).reshape(2,3,3,3)
w_col = w.reshape(2,-1)
w_col

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
        16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26],
       [27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42,
        43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53]])

In [5]:
b = np.ones((1,2))
b

array([[1., 1.]])

In [6]:
y = np.dot(w_col, x_col)
y

array([[ 10197.,  10548.,  11601.,  11952.,  27045.,  27396.,  28449.,
         28800.,  43893.,  44244.,  45297.,  45648.],
       [ 25506.,  26586.,  29826.,  30906.,  77346.,  78426.,  81666.,
         82746., 129186., 130266., 133506., 134586.]])

In [7]:
y = y.T + b
y

array([[ 10198.,  25507.],
       [ 10549.,  26587.],
       [ 11602.,  29827.],
       [ 11953.,  30907.],
       [ 27046.,  77347.],
       [ 27397.,  78427.],
       [ 28450.,  81667.],
       [ 28801.,  82747.],
       [ 43894., 129187.],
       [ 44245., 130267.],
       [ 45298., 133507.],
       [ 45649., 134587.]])

In [8]:
y.reshape(3,2,2,2).transpose(0,3,1,2)

array([[[[ 10198.,  10549.],
         [ 11602.,  11953.]],

        [[ 25507.,  26587.],
         [ 29827.,  30907.]]],


       [[[ 27046.,  27397.],
         [ 28450.,  28801.]],

        [[ 77347.,  78427.],
         [ 81667.,  82747.]]],


       [[[ 43894.,  44245.],
         [ 45298.,  45649.]],

        [[129187., 130267.],
         [133507., 134587.]]]])

# 3 convolutional層

In [9]:
class Conv:
    
    def __init__(self, x_c, y_c, fil_size, stride, pad):
        self.x_c, self.y_c = x_c, y_c
        self.fil_h, self.fil_w = fil_size, fil_size
        self.stride, self.pad = stride, pad
        
        self.w = np.arange(54).reshape(2,3,3,3)
        self.b = np.zeros((1,self.y_c))
        #self.w = np.random.randn(self.y_c, self.x_c, self.fil_h, self.fil_w)
        #self.b = np.random.randn(1, self.y_c)
        
    def forward(self, x):
        
        self.x_b, self.x_c, self.x_h, self.x_w = x.shape
        self.y_h = (self.x_h - self.fil_h + 2*self.pad) // self.stride + 1
        self.y_w = (self.x_w - self.fil_w + 2*self.pad) // self.stride + 1
        
        self.x_col = im2col(x, self.fil_h, self.y_h, self.stride, self.pad)
        self.w_col = self.w.reshape(self.y_c, self.x_c*self.fil_h*self.fil_w)
        
        y = np.dot(self.w_col, self.x_col).T + self.b
        self.y = y.reshape(self.x_b, self.y_h, self.y_w, self.y_c).transpose(0,3,1,2)
        
        return self.y

In [10]:
conv = Conv(3,2,3,1,0)
conv.forward(x)

array([[[[ 10197.,  10548.],
         [ 11601.,  11952.]],

        [[ 25506.,  26586.],
         [ 29826.,  30906.]]],


       [[[ 27045.,  27396.],
         [ 28449.,  28800.]],

        [[ 77346.,  78426.],
         [ 81666.,  82746.]]],


       [[[ 43893.,  44244.],
         [ 45297.,  45648.]],

        [[129186., 130266.],
         [133506., 134586.]]]])